### Having a list of websites get the needed information from it and store into a pandas dataframe

In [12]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

### Load the websites excel file into a dataframe

In [13]:
import pandas as pd

file = 'organisations_website.xlsx'

#load excel
load_excel = pd.ExcelFile(file)

# Print the sheet names
print(load_excel.sheet_names)
print(type(load_excel))

# Load a sheet into a DataFrame
sites_df = load_excel.parse('Sheet1')

sites_df.head(20)

['Sheet1']
<class 'pandas.io.excel.ExcelFile'>


,Organisation name,Website,Country
0,Academy of Physical Education in Wroclaw / Aka...,www.awf.wroc.pl,Poland
1,Academy of Scientific and Innovative Research,www.acsir.res.in,India
2,Academy of Special Education Maria Grzegorzews...,www.aps.edu.pl,Poland
3,Accademia di Belle Arti di Brera,www.accademiadibrera.milano.it,Italy
4,Accademia Nazionale di Santa Cecilia,www.santacecilia.it,Italy
5,Accra Polytechnic,www.apoly.edu.gh,Ghana
6,Acharya Institute of Technology,www.acharya.ac.in,India
7,Acharya N G Ranga Agricultural University,www.angrau.ac.in,India
8,Acharya Nagarjuna University Center for Distan...,www.anucde.info,India
9,Acharya Nagarjuna University Center for Distan...,www.anuonline.ac.in,India


### Define functions to open,close and get an error for websites

In [14]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

### Have choosen a website in English to work with in this notebook

In [15]:
#get the html data
page = ''
page = simple_get('http://'+ sites_df.loc[17,'Website'])

#load into a soup object to work with
soup = BeautifulSoup(page, 'html.parser')
print(type(soup))
print(soup)
#print(sites_df.loc[0,'Website'])

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="/favicon.ico" rel="shortcut icon" type="image/ico"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- <meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" /> -->
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
	@import url(/common/styles/default.css);
	@import url(/common/styles/other.css);
	
	/* testing without cf logic to see what happens */
		@import url(/common/styles/colorbox.css);
	
	/* updates for colorbox.css backgrounds */
	
</style>
<link href="/common/styles/responsive.css" rel="stylesheet"/>
<!--[if IE 7]>
	<style type="text/css">
		@import url(/common/styles/ie7.css);
    </style>
<![endif]-->
<style type="text/css">
	#cbox

### Get the data from the first page about the program names and cities

In [16]:
import unicodedata

#intermediate lists to work with
city_programme = []


all_programmes = soup.find_all('span')
print(type(all_programmes))

# for all the values that has a |, will be stored and a list
for i,program in enumerate(all_programmes):
    if "|" in program.text:
        #print(program.text)
        city_programme.append(unicodedata.normalize("NFKD",program.text))
print(city_programme)


<class 'bs4.element.ResultSet'>
['Chicago  |  Doctor of Psychology', 'Chicago  |  Doctor of Philosophy', 'Chicago  |  Doctor of Philosophy', 'Chicago  |  Doctor of Philosophy', 'Chicago  |  Master of Arts', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Arts in Counseling', 'Chicago  |  Master of Public Administration', 'Chicago  |  Master of Public Policy', 'Vancouver  |  Doctor of Psychology', 'Vancouver  |  Master of Arts', 'Vancouver  |  Master of Arts', "Vancouver  |  Master's", "Vancouver  |  Master's", "Vancouver  |  Master's", 'Vancouver  |  Master of Arts', 'Vancouver  |  Master of Public Policy & Administration', 'Vancouver  |   Master of Public Policy & Administration', 'Online  |  Doctor of Philosophy', 'Online  |  Master of Arts', 'Online  |  Master of Arts', 'Online  |  Master of

In [17]:
df = pd.DataFrame(columns=['city','program_name'])

#split the string into two fields and save in a dataframe
for i in range(len(city_programme)):
    list_s = []
    list_s = city_programme[i].split('|')

    df.loc[i,'city'] = list_s[0]
    df.loc[i,'program_name'] = list_s[1]

#remove duplicated rows
df_new = df.drop_duplicates()
df.describe()
df_new.describe()

#save the final dataframe to a csv file
df_new.to_csv('programs_city.csv',sep= ',', header = ['City','ProgramName'])
df_new

,city,program_name
0,Chicago,Doctor of Psychology
1,Chicago,Doctor of Philosophy
4,Chicago,Master of Arts
5,Chicago,Master of Arts in Counseling
11,Chicago,Master of Public Administration
12,Chicago,Master of Public Policy
13,Vancouver,Doctor of Psychology
14,Vancouver,Master of Arts
16,Vancouver,Master's
20,Vancouver,Master of Public Policy & Administration


### Get the second page of the website to analyze the tuition for programmes

In [18]:
#get the html response and create a Soup object
page_2 = ''
page_2 = simple_get('https://www.adler.edu/page/campuses/chicago/financial-aid/cost-of-attendance/tuition-and-fees')

soup_2 = BeautifulSoup(page_2, 'html.parser')
print(type(soup_2))
#print(soup_2)

#extracting data from a table and find all rows from it
all_rows = soup_2.find_all('tr')

#create intermediate lists and the final dataframe
list_f = []
list_final = []
df_t = pd.DataFrame(columns=['prog_name','price'])


#create the list of arrays with program name and price
for i,tr in ( enumerate(all_rows)):
    tds = unicodedata.normalize("NFKD",tr.text).strip().split('\n')
    list_f = []
    for i in range(len(tds)):
        if tds[i] is not None and tds[i] != '':
            list_f.append(tds[i])
    list_final.append(list_f)

#save in the dataframe 2 fields, program name and price
i = 0
for item in list_final:
    if len(item) >= 2:
        df_t.loc[i,'prog_name'] = item[0]
        df_t.loc[i,'price'] = item[1]
        i += 1
        
#save the dataframe into a csv file for other analysis        
df_t.to_csv('program_price.csv',sep= ',', header = ['ProgramName','Price'])
df_t

<class 'bs4.BeautifulSoup'>


,prog_name,price
0,Admissions Application Fee,$30
1,Deposit (nonrefundable) - M.A. Programs,$200
2,Deposit (nonrefundable) - Doctoral Programs,$300
3,Deferment Fee,$500
4,Clinical M.A. Programs Credit Hour,"$1,265"
5,Non-Clinical M.A. Programs Credit Hour,$855
6,Online Ph.D. Programs Credit Hour,$855
7,Psy.D. Credit Hour,"$1,470"
8,Ph.D. Credit Hour,"$1,420"
9,Audit Clinical M.A. Credit Hour,$630
